# 第7章: データベース
artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．  
（略）  
artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

---
## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import leveldb
import gzip
import json

In [47]:
db = leveldb.LevelDB("07DB")

with gzip.open("artist.json.gz", "rt", "utf-8") as data:
    for line in data:
        artist = json.loads(line)
        name = artist["name"] + "\t " + str(artist["id"])
        area = artist.get("area", "不明")
        db.Put(name.encode(), area.encode())
        
print(f"登録数:{len(list(db.RangeIter()))}")

登録数:921337


---
## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [48]:
import re

#db = leveldb.LevelDB("07DB")
artist = input("artist: ")    #探す人の名前
hit = False
pattern = re.compile(r"(?P<name>[^\t]+)\t(?P<id>.*)")

for key, value in db.RangeIter():
    match = pattern.match(key.decode())
    if match.group("name") == artist:
        name, area, hit = match.group("name"), value.decode(), True
        print(area)
        
if hit == False:
    print("この歌手は登録されていません")

artist: Oasis
United Kingdom
United States
United Kingdom


---
## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [24]:
count, country = 0, "Japan"
for key, value in db.RangeIter():
    if value == country.encode():
        count += 1
print(f"{country}:{count}件")

Japan:22821件


---
## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [27]:
#db = leveldb.LevelDB("63DB")

with gzip.open("artist.json.gz", "rt", "utf-8") as data:
    for line in data:
        artist = json.loads(line)
        name = artist["name"] + "\t " + str(artist["id"])
        area = artist.get("tags", "不明")
        db.Put(name.encode(), json.dumps(area).encode())
        
print(f"登録数:{len(list(db.RangeIter()))}")

登録数:921337


In [46]:
#db = leveldb.LevelDB("07DB")
artist = input("タグを調べる歌手:")    #探す人の名前
hit1, hit2 = False, False
pattern = re.compile(r"(?P<name>[^\t]+)\t(?P<id>.*)")

for key, value in db.RangeIter():
    match = pattern.match(key.decode())
    if match.group("name") == artist :
        name, tags, hit1 = match.group("name"), json.loads(value.decode()), True
        if type(tags) == list:
            for tag in tags :
                hit2 = True
                tag_value, tag_count = tag["value"], tag["count"]
                print(f"{tag_value}:{tag_count}")
        
if hit1 == False:
    print("この歌手は登録されていません")
elif hit2 == False:
    print("この歌手のタグはありません")

タグを調べる歌手:Oasis
rock:1
britpop:3
british:4
uk:1
britannique:1
rock and indie:1
england:1
manchester:1
morning glory:1
oasis:1


---
## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [16]:
import gzip
import json
import pymongo
from pymongo import  MongoClient
"""
client = MongoClient()
db = client.test_db
collection = db.artist

with gzip.open("artist.json.gz", "rt", "utf-8") as data:
    documents = []
    for (i, line) in enumerate(data):
        documents.append(json.loads(line))
        
        if i%100000 == 0:
            collection.insert_many(documents)
            documents = []
            print(f"{i}件登録")
            
    if documents != []:
        collection.insert_many(documents)
        print(f"{i}件登録")
"""

collection.create_index([('name', pymongo.ASCENDING)])  
collection.create_index([('aliases.name', pymongo.ASCENDING)])  
collection.create_index([('tags.value', pymongo.ASCENDING)])
collection.create_index([('rating.value', pymongo.ASCENDING)])

'rating.value_1'

In [15]:
for index in db.artist.list_indexes():
    print(index)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'test_db.artist')])
SON([('v', 2), ('key', SON([('name', 1)])), ('name', 'name_1'), ('ns', 'test_db.artist')])
SON([('v', 2), ('key', SON([('aliases.name', 1)])), ('name', 'aliases.name_1'), ('ns', 'test_db.artist')])
SON([('v', 2), ('key', SON([('tags.value', 1)])), ('name', 'tags.value_1'), ('ns', 'test_db.artist')])
SON([('v', 2), ('key', SON([('rating.value', 1)])), ('name', 'rating.value_1'), ('ns', 'test_db.artist')])


---
## 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

**ターミナルにて**  
\> use test_db  
switched to db test_db   
\> db.artist.find({"name":"Queen"});  
を入力

In [ ]:
client = MongoClient()
db = client.test_db
collection = db.artist

artist = input(f"検索したいアーティスト :")
results = collection.find({"name":artist})

for result in results:
    del result ["_id"]
    print(json.dumps(result, indent='\t', sort_keys=True,))
    print("---------------------------------")

---
## 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ

**ターミナルにて**  
\> use test_db  
switched to db test_db   
\> db.artist.find({"area":"Japan"}).count();  
を入力

---
## 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [ ]:
client = MongoClient()
db = client.test_db
collection = db.artist

artist = input(f"検索したいアーティスト :")
results = collection.find({"aliases.name":artist})

for result in results:
    del result ["_id"]
    print(json.dumps(result, indent='\t', sort_keys=True,))
    print("---------------------------------")

---
## 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [51]:
client = MongoClient()
db = client.test_db
collection = db.artist

results = collection.find({"tags.value":"dance"}).sort("rating.count", pymongo.DESCENDING)
i = 0
for result in results:
    print(json.dumps(result["name"]))
    i += 1
    if i == 10:
        break

"Madonna"
"Bj\u00f6rk"
"The Prodigy"
"Rihanna"
"Britney Spears"
"Maroon 5"
"Adam Lambert"
"Fatboy Slim"
"Basement Jaxx"
"Cornershop"
